In [1]:
# Importing the libraries needed
import os
import re
import sys
from string import punctuation
sys.path.insert(0, "..")

import pandas as pd
import numpy as np

import nltk
import nltk.data
from nltk.sentiment import vader
from nltk.corpus import stopwords

from gensim import corpora
from gensim import models
import enchant
from enchant.checker import SpellChecker
from tqdm import tqdm_notebook as tqdm

In [2]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [4]:
# Retrieved saved files
df = pd.read_csv("cleaned_df.csv")

In [5]:
true = df[:500].copy()
false =  df[501:1000].copy()
# Use test for testing
test =  df[1001:].copy()
true["label"] = 'true'
false["label"] = 'false'
# Use test for testing
test["label"] = ''

In [6]:
train = pd.concat([true, false])

In [7]:
train.head(2)

,Unnamed: 0,Unnamed: 0_x,type,text,article_name,date,index_article,article_filepath,dir,Unnamed: 0_y,...,newspaper_volume,newspaper_issuenumber,newspaper_language,count,non_jkc,text_clean,text_no_point,text_clean_split,text_clean_split_no_point,label
0,0,36288,p,Aoiang er mijnen bestaan is het mijngas de gro...,DDD_010417712_0100_articletext.xml,1950-11-16,113601,../data/1950/11-16/DDD_010417712/DDD_010417712...,../data/1950/11-16/DDD_010417712,882.0,...,33.0,269.0,nl,28,True,Aoiang er mijnen bestaan is het mijngas de gro...,Aoiang er mijnen bestaan is het mijngas de gro...,['Aoiang er mijnen bestaan is het mijngas de g...,['Aoiang er mijnen bestaan is het mijngas de g...,true
1,1,36381,p,ning tijdens de ontgassing normaal kan doorgaa...,DDD_010417712_0102_articletext.xml,1950-11-16,113628,../data/1950/11-16/DDD_010417712/DDD_010417712...,../data/1950/11-16/DDD_010417712,882.0,...,33.0,269.0,nl,20,True,ning tijdens de ontgassing normaal kan doorgaa...,ning tijdens de ontgassing normaal kan doorgaa...,['ning tijdens de ontgassing normaal kan doorg...,['ning tijdens de ontgassing normaal kan doorg...,true


You are interested in `train["text_no_point"]` and `train["text_clean"]` (or train.text_no_point). Which is the column that contains all the selected cleaned texts.

What we would like to do is to select similar texts based on a golden sample of texts. 
- Which means that we will go through all our dataset and select based on the frequency of the appearance of certain words (oil, gas and coal)
- After that the results are ranked based on the number of matches of these words and these synonyms.
- Once that is done, you have the `df` at hand in this notebook.

We assume that the first 500 rows of this dataframe are the "true" labelled text representing texts we want to find similar texts to. And the next 500 are the false ones.

### A small start

I deleted the small start as I gave you a dataset in which most of the texts are already "cleaned". the `text_no_point` contains the text without any punctuation. The `text_clean` contain text with punctuation. I suggest you to start from `text_no_point`


From here apply the tf-idf/BM 25 or other similarity algorithms!
- Use the documentation (and internent examples) from the following libraries:
    - nltk --> for the already-written-out main algorithms. This will be your main library with:
    - gensim
    - Enchant (might be useful)